In [1]:
import math

import cvxpy as cp
import numpy as np


# %%
def PriceReverse(df, cycle, time):
    """
    Compute 1M Price Reversal
    Order: Ascending
    :param df: dataframe object (n*1 vector)
    :param cycle: how many days to look back to see its reversal
    :param time: current index for df to look at
    :return: PM_{i,t} = (Close_{i,t} - Close_{i, t-1}) / Close_{i, t-1}
    """
    try:
        previous_price = df.iloc[time - cycle]
        return (df.iloc[time] - previous_price) / previous_price
    except KeyError:
        pass


def PriceMomentum(df, cycle, time):
    """
    Compute 1M Price Reversal
    Order: Descending
    :param df: dataframe object (n*1 vector)
    :param cycle: how many days to look back to see its reversal
    :param time: current index for df to look at
    :return: PM_{i,t} = (Close_{i,t} - Close_{i, t-1}) / Close_{i, t-1}
    """
    try:
        previous_price = df.iloc[time - 2*cycle]
        return -(df.iloc[time] - previous_price) / previous_price
    except KeyError:
        pass


def Price_High_Low(df, cycle, time):
    """
    Compute High-minus-low:
    Order: Descending
    :param df: dataframe object (n*1 vector)
    :param cycle: how many days to look back to see its reversal
    :param time: current index for df to look at
    :return: HL_{i,t} = (High_{i,t} - Close_{i,t}) / (Close_{i,t} - Low_{i,t})
    """
    try:
        High = max(df.iloc[time - cycle:time])
        Low = min(df.iloc[time - cycle:time])
        return -(High - df.iloc[time]) / (df.iloc[time] - Low)
    except KeyError:
        pass


def Vol_Coefficient(df, cycle, time):
    """
    Compute Coefficient of Variation:
    Order: Descending
    :param df: dataframe object (n*1 vector)
    :param cycle: how many days to look back to see its reversal
    :param time: current index for df to look at
    :return: CV_{i,t} = Std(Close_i, cycle) / Ave(Close_i, cycle)
    """
    try:
        data = df.iloc[time - cycle:time].pct_change(1).dropna()
        std = np.std(data)
        avg = np.mean(data)
        return -std / avg
    except KeyError:
        pass


def AnnVol(df, cycle, time):
    """
    Compute Annual Volatility:
    Order: Descending
    :param df: dataframe object (n*1 vector)
    :param cycle: how many days to look back to see its reversal
    :param time: current index for df to look at
    :return: AnnVol = sqrt(252) * sqrt(1/21 * sum(r_{i,t-j}^2))
    where r_{i,s} = log(Close_{i,t} / Close_{i,t-1})
    """
    try:
        r_2 = int(0)
        for i in range(1, cycle):
            log = np.log(df.iloc[time - i] / df.iloc[time - i - 1])
            r_2 += log ** 2
        result = np.sqrt(252 / cycle * r_2)
        return -result
    except KeyError:
        pass


def MACD(df, cycle, time):
    """
    Compute Moving Average Convergence Divergence:
    Order: Descending
    :param df: dataframe object (n*1 vector)
    :param cycle: how many days to look back to see its reversal
    :param time: current index for df to look at
    :return: cycle-Period EMA - cycle*2-Period EMA
    where EMA = Price_t * k + EMA_t-1 * (1-k)
    k = 2 / (N+1)
    """
    try:
        data = df.iloc[time - cycle:time]
        EMA_SR = data.ewm(span=cycle).mean()
        EMA_LR = data.ewm(span=cycle*2).mean()
        res = list(EMA_SR)[-1] - list(EMA_LR)[-1]
        return res
    except KeyError:
        pass


def BoolingerBands(df, cycle, time):
    """
    Compute Boolinger Bands:
    Order: Descending
    :param df: dataframe object (n*1 vector)
    :param cycle: how many days to look back to see its reversal
    :param time: current index for df to look at
    :return: Ave(cycle) +- 2 * Std(cycle)
    """
    if time - 2 * cycle <= 0 and not math.isnan(df.iloc[time - cycle]):
        return 0
    try:
        data_lr = df.iloc[time - 2*cycle+1:time]
        data_sr = df.iloc[time - cycle:time]
        SMA = data_lr.rolling(cycle).mean()
        SMA.dropna(inplace=True)
        delta = np.std(data_sr)
        up_bound = SMA + delta
        lw_bound = SMA - delta
        midpoint = len(data_sr) // 2
        res = sum(data_sr[:midpoint] > up_bound[:midpoint]) - sum(data_sr[midpoint:] < lw_bound[midpoint:])
        return res * np.std(data_sr.pct_change(1).dropna())
    except ValueError:
        pass



trading_strategies = [PriceReverse, PriceMomentum, Price_High_Low, Vol_Coefficient, AnnVol, MACD, BoolingerBands]


In [2]:
def MinVariance(data, ranking, time, cycle):
    """
    MinVariance minimizes variance (needs short positions)
    Argument ranking: list of stocks from PitchStock
            return weighting for each stock (in percentage)
    """
    covar = np.zeros(shape=(len(ranking), cycle))
    for i in range(len(ranking)):
        covar[i] = data[ranking[i]].iloc[time-cycle:time].fillna(method='Backfill')
    inv_cov_matrix = np.linalg.pinv(np.cov(covar))
    ita = np.ones(inv_cov_matrix.shape[0])
    weight = (inv_cov_matrix @ ita) / (ita @ inv_cov_matrix @ ita)
    return weight


def EqualWeight(data, ranking, time, cycle):
    """
    EqualWeight assign weight by 1/N
    return weighting for each stock (in percentage)
    """
    N = len(ranking)
    weight = np.ones(shape=N) / N
    return weight


def MeanVariance_Constraint(data, ranking, time, cycle):
    """
    Mean Variance solved by convex optimization
    return weighting for each stock (in percentageg)
    """
    covar = np.zeros(shape=(len(ranking), cycle))
    for i in range(len(ranking)):
        covar[i] = data[ranking[i]].iloc[time-cycle:time].fillna(method='Backfill')
    cov_matrix = np.cov(covar)
    weight = cp.Variable(shape=len(ranking))
    objective = cp.Minimize(cp.quad_form(weight, cov_matrix))
    constraints = [cp.sum(weight) == 1, weight >= 1 / (2 * len(ranking))]
    problem = cp.Problem(objective, constraints)
    result = problem.solve()
    return weight.value


def RiskParity(data, ranking, time, cycle):
    """
    RiskParity inversely invest for stock according to their volatility
    disregards covariance is the major drawback
    return weighting for each stock (in percentage)
    """
    covar = np.zeros(shape=(len(ranking), cycle))
    for i in range(len(ranking)):
        covar[i] = data[ranking[i]].iloc[time - cycle:time]
    vol = np.array(covar.std(axis=1))
    vol = np.reciprocal(vol)
    weight = vol / vol.sum()
    return weight


rebalancing_strategies = [MinVariance, EqualWeight, MeanVariance_Constraint, RiskParity]
# rebalancing_strategies = [EqualWeight, RiskParity]

In [4]:
import copy

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

#from Strategy import *

sns.set()


# %%
class Agent:

    def __init__(self, portfolio, data, trading_strategies, rebalancing_strategies, cycle, max_holding):
        """
        portfolio: dictionary (accounting book)
        Max_holding is the maximum number of stocks ths agent can hold
        Cycle is the rebalancing period
        Data is the dataset
        Strategies is which factor investing stratgy this Agent has in disposal
        """
        self.portfolio = portfolio
        self.data = data
        self.trading_strategies = trading_strategies
        self.rebalancing_strategies = rebalancing_strategies
        self.cycle = cycle
        self.equity = INITIAL_BALANCE
        self.re = float()
        self.tran_cost = float()
        self.rf = np.power(RISKFREE, self.cycle / 252)
        self.max_holding = max_holding

    def PitchStock(self, trading_strategy, time):
        """
        Argument trading_strategy: a function that takes (df, cycle, time) as argument
        return ranking: list of stocks that should invest
        """
        cycle = self.cycle
        data = self.data
        max_holding = self.max_holding
        ranking = {}
        for i in ticker:
            metric = trading_strategy(data[i], cycle, time)
            if metric is not None and not math.isnan(metric):
                ranking[i] = trading_strategy(data[i], cycle, time)
        result = sorted(ranking, key=ranking.get)[:max_holding]
        return result

    def Rebalancing(self, ranking, rebalancing_strategy, time):
        """
        Argument ranking: result from Agent.PitchStock
                rebalancing_strategy: a function that takes (df, ranking, time, cycle) as argument
                return target_portfolio: dictionary {Stock: # of shares}
        """
        cycle = self.cycle
        data = self.data
        cash = self.portfolio['cash']
        # assume that cash earns risk-free rate interest
        equity = self.get_Equity(time) + cash * (self.rf - 1)
        target_portfolio = {}
        weight = np.array(rebalancing_strategy(data, ranking, time, cycle))
        weight = (weight * equity).astype(int)
        for w, stock in zip(weight, ranking):
            price = data[stock].iloc[time]
            shares = w // price
            target_portfolio[stock] = shares
            equity -= shares * price
        target_portfolio['cash'] = equity
        return target_portfolio

    def get_Equity(self, time):
        """
        return the equity value for a given time
            sum weight * price
        """
        data = self.data
        portfolio = copy.deepcopy(self.portfolio)
        cash = portfolio['cash']
        total_equity = cash
        # compute the stock value
        del portfolio['cash']
        ticker = list(portfolio)
        shares = np.array(list(portfolio.values()))
        print(shares)
        price = np.matrix(data[ticker].iloc[time])
        print(price)
        total_equity += price @ shares
        return total_equity.item()

    def Trading(self, target_portfolio, time):
        """
        Argument target_portfolio: a dictionary get from rebalance 
                    (what Agent.portfolio should be after trading)
        returns nothing but update:
                equity, portfolio, re, tran_cost
        """
        # take all necessary attributes from the class
        cost = 0
        portfolio = self.portfolio
        # selling and adjust share
        for i in list(portfolio):
            if i not in target_portfolio and i != 'cash':
                cost += portfolio[i] * TRANS_COST
            elif i in target_portfolio and i != 'cash':
                diff = abs(portfolio[i] - target_portfolio[i])
                cost += diff * TRANS_COST
        # buying
        for i in target_portfolio:
            if i not in portfolio:
                cost += target_portfolio[i] * TRANS_COST
        # update all the attribute of the agent
        self.tran_cost += cost
        self.portfolio = target_portfolio
        self.equity = self.get_Equity(time) - cost
        self.re = self.equity / INITIAL_BALANCE

    def get_Vol(self, time):
        """
        use portfolio weights to calcualte equity paths in this cycle
        and then compute its variance
        """
        cycle = self.cycle
        data = self.data
        portfolio = copy.deepcopy(self.portfolio)
        del portfolio['cash']
        # this is a vector of max_holding number of elements
        shares = np.array(list(portfolio.values()))
        # ticker in the portfolio except cash
        ticker = list(portfolio)
        price_matrix = np.matrix(data[ticker].iloc[time + 1 - cycle:time + 1])
        equity_path = price_matrix @ shares
        return equity_path

    def BackTesting_Single(self, trading_strategy, rebalancing_strategy):
        """
        This is backtsting for one single combination of trading and rebalancing strategy
        Return the total return, volatility and Sharpe ratio
        """
        cycle = self.cycle
        data = self.data
        print("Trading strategy: %s" % trading_strategy.__name__)
        print("\n")
        print("Rebalancing strategy: %s" % rebalancing_strategy.__name__)
        print("\n")
        T = len(data) // cycle
        print("We are rebalancing for %s number of times." % T)
        portfolio_path = []
        for i in range(1, T):
            time = i * cycle
            ranking = self.PitchStock(trading_strategy, time)
            target_portfolio = self.Rebalancing(ranking, rebalancing_strategy, time)
            print(time)
            print(self.re)
            # get volatility before portfolio updates
            portfolio_path.append(self.get_Vol(time))
            self.Trading(target_portfolio, time)
            print("Rebalancing for %s time!" % i)
        vol = np.std(portfolio_path) / np.sqrt(T * cycle) / 100
        # annualized return
        annual_return = (np.power(self.re, 252 // cycle / T) - 1) * 100
        # annualized risk free
        total_rf = np.power(RISKFREE, cycle * T / 252)
        sharpe = (self.re - total_rf) / vol
        return annual_return, vol, sharpe

    def BackTesting(self):
        """
        This is backtsting for all strategies
        Return two dictionary
            1. return for each strategy
            2. overall cost for each strategy
        """
        trading_strategies = self.trading_strategies
        rebalancing_strategies = self.rebalancing_strategies
        print("There are %s trading strategies and %s rebalancing strategies we are testing." % (
            len(trading_strategies), len(rebalancing_strategies)))
        print("They are: ")
        for i in trading_strategies:
            print("     %s" % i.__name__)
        print('\n')
        for i in rebalancing_strategies:
            print("     %s" % i.__name__)
        portfolio_re = pd.DataFrame(index=[x.__name__ for x in rebalancing_strategies],
                                    columns=[x.__name__ for x in trading_strategies])
        portfolio_vol = pd.DataFrame(index=[x.__name__ for x in rebalancing_strategies],
                                     columns=[x.__name__ for x in trading_strategies])
        portfolio_sharpe = pd.DataFrame(index=[x.__name__ for x in rebalancing_strategies],
                                        columns=[x.__name__ for x in trading_strategies])
        for col, trading_strategy in enumerate(trading_strategies):
            for row, rebalancing_strategy in enumerate(rebalancing_strategies):
                # use BackTesting_Single to get the three value of metrics needed
                total_return, vol, sharpe = self.BackTesting_Single(trading_strategy, rebalancing_strategy)
                portfolio_re.iloc[row][col] = total_return
                portfolio_vol.iloc[row][col] = vol
                portfolio_sharpe.iloc[row][col] = sharpe
                # reset balance, equity, re, and transaction cost for the agent
                self.reset()
                print("\n")
        # turn this dictionary into a nicely presentable dataframe
        return portfolio_re, portfolio_vol, portfolio_sharpe

    def reset(self):
        """
        This reset the Agent to its initial holding. 
        Apply this method between testing different strategies.
        """
        self.portfolio = {'cash': INITIAL_BALANCE}
        self.equity = INITIAL_BALANCE
        self.re = float()
        self.tran_cost = float()


In [5]:
df = pd.read_csv("SP500.csv")
df.drop(['Unnamed: 0'], axis=1, inplace=True)
print(df.shape)
ticker = list(df.columns)[1:]
# rebalance portfolio every month (20 trading days)
INITIAL_BALANCE = 51500
TRANS_COST = 0.00
# define the risk-free rate
RISKFREE = 1.00
wsw = Agent({'cash': INITIAL_BALANCE}, df, trading_strategies, rebalancing_strategies, 20, 10)

(4783, 456)


In [6]:
wsw.portfolio

{'cash': 51500}

In [167]:
wsw.BackTesting_Single(trading_strategies[0], EqualWeight)

Trading strategy: PriceReverse


Rebalancing strategy: EqualWeight


We are rebalancing for 238 number of times.
[]
[]
20
0.0
[585. 251. 466. 285. 261. 581. 388. 401. 847. 123.]
[[ 8.8  20.44 11.05 18.07 19.7   8.85 13.26 12.82  6.08 41.86]]
Rebalancing for 1 time!
[585. 251. 466. 285. 261. 581. 388. 401. 847. 123.]
[[ 8.64 20.07 12.72 19.91 20.28  8.83 13.32 14.7   7.47 47.57]]
40
0.9999999999999999
[1428.  427.   66.  235.  732.  746.  205.  100.  325.  330.]
[[ 3.88 12.97 83.9  23.5   7.56  7.42 26.93 55.34 17.   16.77]]
Rebalancing for 2 time!
[1428.  427.   66.  235.  732.  746.  205.  100.  325.  330.]
[[ 2.75 14.7  37.38 15.52  7.25  5.33 14.23 35.28  8.25 14.22]]
60
1.0759753398058252
[ 105.  478. 1091.  277.  218.  440.  144.  316.  332.  308.]
[[37.38  8.25  3.62 14.23 18.07  8.96 27.39 12.47 11.89 12.81]]
Rebalancing for 3 time!
[ 105.  478. 1091.  277.  218.  440.  144.  316.  332.  308.]
[[34.56  7.91  3.95 19.07 32.97  9.95 44.08 18.95 12.16 13.93]]
80
0.767211067961165
[

[2220.  513. 2149. 1082.  718. 1393.  478.  772. 3061. 1217.]
[[ 4.22 20.35  5.9  10.1  15.18  7.22 21.9  12.25  3.86  7.78]]
660
1.8367019417475736
[2352. 1357.  748.  702.  470. 1687. 2490. 2831. 1896.  792.]
[[ 4.49  7.78 14.11 15.04 22.46  6.26  4.24  3.73  5.57 13.33]]
Rebalancing for 33 time!
[2352. 1357.  748.  702.  470. 1687. 2490. 2831. 1896.  792.]
[[ 5.15 10.57 15.58 14.02 23.81  7.08  5.12  4.38  6.09 14.96]]
680
2.050801165048544
[1323. 1092. 1086. 1821.  289. 3014. 1664.  872.  587. 1675.]
[[ 9.04 10.95 11.01  6.57 41.34  3.97  7.19 13.72 20.38  7.14]]
Rebalancing for 34 time!
[1323. 1092. 1086. 1821.  289. 3014. 1664.  872.  587. 1675.]
[[ 9.02 11.57 11.04  6.29 37.28  3.66  7.26 13.2  19.37  7.67]]
700
2.323576699029127
[ 905. 1612.  386. 2179. 2302.  765. 1910.  399.  913.  360.]
[[13.    7.3  30.45  5.4   5.11 15.37  6.16 29.42 12.88 32.6 ]]
Rebalancing for 35 time!
[ 905. 1612.  386. 2179. 2302.  765. 1910.  399.  913.  360.]
[[12.98  7.55 17.9   6.11  6.19 16.74  6

[ 863.  537. 1371.  976.  937.  604. 6359. 1824.  730.  533.]
[[26.58 42.93 20.87 24.75 25.33 42.25  4.12 12.91 37.29 43.16]]
1320
4.161914299029125
[6798.  681. 1581.  309. 1057. 2855.  713. 1081. 1093.  646.]
[[ 3.65 36.41 15.69 80.19 23.46  8.69 34.78 22.94 22.69 38.39]]
Rebalancing for 66 time!
[6798.  681. 1581.  309. 1057. 2855.  713. 1081. 1093.  646.]
[[ 4.21 35.59 15.17 76.9  22.6   8.15 33.95 27.67 22.42 37.42]]
1340
4.818105755339806
[ 842.  650. 1070.  512.  630. 1451.  551. 1080. 2345. 2504.]
[[29.74 38.52 23.42 48.94 39.74 17.27 45.47 23.21 10.69 10.01]]
Rebalancing for 67 time!
[ 842.  650. 1070.  512.  630. 1451.  551. 1080. 2345. 2504.]
[[32.13 47.68 21.26 44.23 40.13 16.84 41.61 20.05 11.32  9.52]]
1360
4.867787697087379
[ 918. 1312. 1696. 2883. 4441.  908. 1003. 1032.  856.  951.]
[[27.04 18.91 14.63  8.61  5.59 27.34 24.74 24.04 29.   26.1 ]]
Rebalancing for 68 time!
[ 918. 1312. 1696. 2883. 4441.  908. 1003. 1032.  856.  951.]
[[29.01 19.82 13.07  9.11  5.08 28.43 

[  81. 1004. 1747.  454. 1092. 1213.  372.  878. 1716.  651.]
[[69.35 17.51 13.01 48.1  20.96 19.04 66.41 27.67 11.92 35.63]]
1940
4.6185283631067895
[   298. 172233.   1264.    924.    809.   1180.   1268.    824.   2726.
   1632.]
[[69.35  0.12 16.35 22.36 25.53 17.51 16.29 25.06  7.58 12.66]]
Rebalancing for 97 time!
[   298. 172233.   1264.    924.    809.   1180.   1268.    824.   2726.
   1632.]
[[33.94  0.07 13.15 16.52 22.45 15.82 12.24 16.86  5.31  9.46]]
1960
4.013242149514557
[ 468. 1141.  620. 9334.  991.  442. 2028.  653. 2439. 1001.]
[[32.08 13.17 24.22  1.61 15.15 33.94  7.41 22.98  6.16 15.  ]]
Rebalancing for 98 time!
[ 468. 1141.  620. 9334.  991.  442. 2028.  653. 2439. 1001.]
[[26.05 10.08 19.13  1.19 10.91 28.06  6.18  7.26  6.79  7.57]]
1980
2.9182260330097036
[1534. 2684.  788.  922. 2320. 1351. 1056. 1471.  760.  207.]
[[ 7.26  4.15 14.13 12.08  4.8   8.24 10.54  7.57 14.65 53.73]]
Rebalancing for 99 time!
[1534. 2684.  788.  922. 2320. 1351. 1056. 1471.  760.  

[5238.  396. 4373. 1305. 1999. 5944. 4582. 1323. 1925. 4311.]
[[  8.45 108.17   9.85  32.87  24.52   8.67   9.06  35.    26.83  11.49]]
2560
9.511300640776692
[1265. 2624. 3275. 1606. 5107. 3360. 3592. 2828. 1029. 1200.]
[[36.57 17.63 14.13 28.8   9.06 13.77 12.88 16.36 44.95 38.55]]
Rebalancing for 128 time!
[1265. 2624. 3275. 1606. 5107. 3360. 3592. 2828. 1029. 1200.]
[[37.24 16.08 14.14 30.18 10.01 14.37 12.8  18.15 40.73 44.75]]
2580
8.985855786407763
[2244.  504. 5554. 3029. 1370. 1797. 2746. 1158. 1602. 1059.]
[[21.23 94.39  8.58 15.73 34.77 26.51 17.35 41.13 29.74 44.97]]
Rebalancing for 129 time!
[2244.  504. 5554. 3029. 1370. 1797. 2746. 1158. 1602. 1059.]
[[ 25.79 100.49  10.04  16.52  33.61  27.51  17.76  43.3   29.74  45.96]]
2600
9.25322297087378
[1848. 1164. 2628.  546.  577.  842. 2298. 1185. 4022.  928.]
[[27.33 43.4  19.22 92.37 87.46 60.   21.98 42.63 12.56 54.42]]
Rebalancing for 130 time!
[1848. 1164. 2628.  546.  577.  842. 2298. 1185. 4022.  928.]
[[24.6  41.81 18

...

D:\GitHub\Backtesting\Strategy.py:19: RuntimeWarning: invalid value encountered in double_scalars
  return (df.iloc[time] - previous_price) / previous_price


[-1.32560720e+08 -6.77012500e+07 -4.84212780e+07 -2.76097160e+07
 -2.59201410e+07 -2.82563638e+08 -7.70259560e+07 -4.37636780e+07
 -4.27870830e+07 -7.33680790e+07]
[[20.36 33.54 52.09 81.33 94.66  8.48 31.08 54.62 51.45 31.24]]
3560
nan
[-1.72919210e+07 -7.65045840e+07 -8.24053590e+07 -1.64810718e+08
 -3.46703860e+07 -4.02301180e+07 -3.40600110e+07 -3.19756360e+07
 -4.13136530e+07 -4.51911550e+07]
[[124.19  28.07  26.06  13.03  61.94  53.38  63.05  67.16  51.98  47.52]]
Rebalancing for 178 time!
[-1.72919210e+07 -7.65045840e+07 -8.24053590e+07 -1.64810718e+08
 -3.46703860e+07 -4.02301180e+07 -3.40600110e+07 -3.19756360e+07
 -4.13136530e+07 -4.51911550e+07]
[[119.43  29.07  25.21  11.31  61.01  54.18  62.76  68.    51.12  47.88]]
3580
nan
[-7.95364315e+08 -1.05114227e+08 -1.08678323e+08 -8.04903920e+07
 -2.73390670e+07 -3.22881320e+07 -5.38081600e+07 -7.70535940e+07
 -1.89874770e+08 -3.55720340e+07]
[[ 2.7  20.43 19.76 26.68 78.55 66.51 39.91 27.87 11.31 60.37]]
Rebalancing for 179 time

[-5.96688990e+07 -3.55543660e+07 -6.72349300e+07 -4.26257180e+07
 -1.28131483e+08 -3.76157590e+07 -3.77148520e+07 -1.14270410e+07
 -7.19907360e+07 -1.42349440e+07]
[[ 34.06  62.41  34.01  51.    19.01  62.06  63.32 208.19  33.27 162.85]]
3900
nan
[-1.02799601e+08 -1.66549070e+07 -8.67670170e+07 -1.49442147e+08
 -7.90097010e+07 -2.84434921e+08 -9.52743420e+07 -6.19764410e+07
 -2.37422190e+07 -9.07643140e+07]
[[ 20.89 128.94  24.75  14.37  27.18   7.55  22.54  34.65  90.45  23.66]]
Rebalancing for 195 time!
[-1.02799601e+08 -1.66549070e+07 -8.67670170e+07 -1.49442147e+08
 -7.90097010e+07 -2.84434921e+08 -9.52743420e+07 -6.19764410e+07
 -2.37422190e+07 -9.07643140e+07]
[[ 21.57 139.82  28.77  13.92  31.     8.31  25.81  41.55  98.75  27.23]]
3920
nan
[-2.44059970e+07 -6.92290030e+07 -2.94579380e+07 -4.34361580e+07
 -1.04597130e+07 -1.67510425e+08 -3.93096040e+07 -2.38080230e+07
 -6.21018990e+07 -3.38613010e+07]
[[ 87.99  31.02  72.9   49.44 205.31  12.82  54.63  90.2   34.58  63.42]]
Reba

[[ 92.96  10.93  13.95 174.02 172.84  30.51  30.79  47.07  89.24  43.95]]
4360
nan
[-1.28284567e+08 -3.44203190e+07 -7.10146710e+07 -2.95349150e+07
 -1.06363730e+07 -4.48701140e+07 -3.82250570e+07 -5.20349810e+07
 -2.59514650e+07 -5.76815380e+07]
[[ 16.74  62.39  30.24  72.71 201.9   47.86  56.18  41.27  82.75  37.23]]
Rebalancing for 218 time!
[-1.28284567e+08 -3.44203190e+07 -7.10146710e+07 -2.95349150e+07
 -1.06363730e+07 -4.48701140e+07 -3.82250570e+07 -5.20349810e+07
 -2.59514650e+07 -5.76815380e+07]
[[ 17.55  67.91  31.93  70.67 218.11  49.61  58.66  43.23  90.35  37.3 ]]
4380
nan
[-1.04348088e+08 -6.80444760e+07 -1.75448011e+08 -4.55941330e+07
 -1.67171390e+07 -2.64208130e+07 -1.00302833e+08 -7.38474440e+07
 -5.01397070e+07 -1.80082490e+07]
[[ 20.58  31.56  12.24  47.1  128.46  81.28  21.41  29.08  42.83 119.25]]
Rebalancing for 219 time!
[-1.04348088e+08 -6.80444760e+07 -1.75448011e+08 -4.55941330e+07
 -1.67171390e+07 -2.64208130e+07 -1.00302833e+08 -7.38474440e+07
 -5.01397070

(nan, nan, nan)

In [15]:
test = df.MMM
MA_20 = list(test.rolling(20).mean())[-1]
MA_20

173.17250000000007

In [187]:
df.AAP.first_valid_index()

226

In [192]:
test = df['BHF'].iloc[226:]

In [193]:
sum(test.isna())

2231

In [196]:
def preprocessing(df):
    del_ticker = []
    for ticker in df.columns[1:]:
        start = df[ticker].first_valid_index()
        dta = df[ticker].iloc[start:]
        if sum(dta.isna()) != 0:
            del_ticker.append(ticker)
    df.drop(del_ticker, axis=1, inplace=True)
    return df

In [197]:
test = preprocessing(df)

In [198]:
test

,Date,MMM,AOS,ABT,ABBV,ACN,ATVI,AYI,ADBE,AAP,...,WLTW,WYNN,XEL,XRX,XLNX,XYL,YUM,ZBH,ZION,ZTS
0,2001-01-02,35.31,0.61,9.48,NaN,NaN,1.05,NaN,23.22,NaN,...,NaN,NaN,12.42,11.29,31.07,NaN,3.79,NaN,45.75,NaN
1,2001-01-03,34.90,0.63,9.27,NaN,NaN,1.13,NaN,28.79,NaN,...,NaN,NaN,11.98,11.57,36.68,NaN,4.00,NaN,47.27,NaN
2,2001-01-04,35.25,0.63,8.42,NaN,NaN,1.24,NaN,26.71,NaN,...,NaN,NaN,11.13,13.50,37.58,NaN,4.01,NaN,46.32,NaN
3,2001-01-05,33.94,0.61,8.47,NaN,NaN,1.23,NaN,25.59,NaN,...,NaN,NaN,11.15,13.08,35.87,NaN,3.88,NaN,45.79,NaN
4,2001-01-08,34.14,0.60,8.39,NaN,NaN,1.22,NaN,24.63,NaN,...,NaN,NaN,11.29,14.05,34.66,NaN,3.86,NaN,45.89,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4770,2019-12-18,169.03,47.13,86.93,89.33,205.76,59.02,138.16,324.38,157.97,...,201.06,138.71,63.38,37.04,96.93,77.91,98.74,148.64,51.35,126.99
4771,2019-12-19,172.15,47.35,87.35,88.77,208.30,59.13,136.48,327.63,156.48,...,203.19,139.84,63.44,36.78,96.29,77.59,99.82,148.56,51.49,128.74
4772,2019-12-20,175.37,47.42,86.66,89.29,211.10,59.22,136.48,327.61,158.12,...,204.98,138.05,63.74,37.10,97.74,78.61,100.59,150.08,51.63,132.68
4773,2019-12-23,178.47,47.14,87.35,90.25,210.83,58.89,137.06,328.95,159.97,...,200.93,140.23,62.62,37.10,98.90,78.95,99.81,151.00,51.34,132.37
